In [1]:
from handcalcs import render 
import forallpeople as si

si.environment('default', top_level=True)

from scipy.optimize import fsolve

import sympy as sp
import numpy as np

from numpy import log as ln
from numpy import pi, arctan, sin, e

from math import ceil as Arredondar
from math import fabs as Absoluto
from math import sqrt

import pandas as pd


In [27]:
#from handcalcs.install_templates import install_html
#install_html(restore=True)

# Enunciado

Determine a potência que o motor de uma laminadora deverá fornecer na deformação:

- Seção quadrada com área de 1200 mm²;

- Rolos de aço, com espaçamento de 25mm (portanto, a espessura da chapa);

- A rotação dos cilindros é da ordem de 40 RPM;

- O diâmetro dos cilindros é 200 mm;

- O rendimento do processo é de 87,5%;

- Composição do material: 0,65%C, 0,80%Mn, 0,2%Cr, 0,11%Si, 0,08%S, 0,10%P; 

- O material é laminado a 1000°C.

### Definindo grandezas sobre a espessura: 

In [2]:
%%render
#long

b = (1200*mm**2)**0.5

h = b

phi_total_h = ln((25*mm)/(b))

phi_h = phi_total_h

<IPython.core.display.Latex object>

Para chegar na configuração final, a espessura deverá se deformar **-32.6%**.

Organizando as espessuras de entrada e de saída da gaiola:

In [3]:
%%render
#short

h_i = h
h_f = h*e**phi_h

<IPython.core.display.Latex object>

In [4]:
%%render

Delta_h = h_i - h_f

<IPython.core.display.Latex object>

### Definindo grandezas de contato:

Para definir o raio do rolo:

In [5]:
%%render
#parameters

Temp_processo = 1000*Celsius

C = 0.0005/Celsius

<IPython.core.display.Latex object>

In [6]:
%%render
#short

mu = 0.8*(1.05-C*Temp_processo) #específico para aços

<IPython.core.display.Latex object>

In [7]:
%%render
#short

Raio = ((200/2)*mm) #configuração da máquina

CritSeg = 1.5 #sendo extremamente conservador

Raio_Minimo = CritSeg * Delta_h/((sin(arctan(mu)))**2)

<IPython.core.display.Latex object>

Mesmo com um CS alto, o raio mínimo requirido é bem menor que o raio presente no maquinário. Não há necessidade de ajuste no projeto.

### Cálculos de contato

In [8]:
%%render
#long

l_d = (Raio*Delta_h)**0.5

<IPython.core.display.Latex object>

In [9]:
%%render
#long

b_i = b

Temp_ref = 1000 #K
Temp_proc = (1000 + 273) #K

C_bmu = Temp_ref/Temp_proc

<IPython.core.display.Latex object>

In [10]:
%%render
#long

phi_b = -phi_h*e**(-C_bmu*(b_i/l_d))

b_f = b_i*e**phi_b

<IPython.core.display.Latex object>

Utilizaremos a média entre a entrada e a saída para os cálculos futuros:

In [11]:
%%render
#short

b_m = (b_i + b_f)/2

<IPython.core.display.Latex object>

E agora, é possível calcular a área de contato $A_c$:

In [12]:
%%render
#short

A_c = b_m*l_d

<IPython.core.display.Latex object>

### O comprimento varia de etapa para etapa, e sua deformação pode ser deduzida da equação de conservação de massa

In [13]:
%%render 
#short 8 

phi_l = -phi_b - phi_h

<IPython.core.display.Latex object>

Não foi pedido pelo exercício, mas podemos estimar qual comprimento final da barra ao fim do processo:

In [24]:
%%render
#long

l_i = (1*m)

l_f = l_i*e**phi_l #para cada 1 metro que entra no processo, 1.2 metros são "gerados"

<IPython.core.display.Latex object>

In [15]:
%%render
#long

V_linear = Raio*40*(2*pi)/60*(1/s)

Tempo = l_f/V_linear

<IPython.core.display.Latex object>

### A tensão de escoamento $\sigma_e$, para laminação a quente, é constante

In [16]:
%%render
#long

Temp_processo=1000 #°C

C_perc = 0.65 #%

Mn_perc = 0.80 #%

Cr_perc = 0.2 #%

sigma_e_kgfmm2 = (14 - 0.01*Temp_processo)*(1.4+C_perc+Mn_perc+0.3*Cr_perc) #kgf/mm²

sigma_e = sigma_e_kgfmm2*9.81*MPa #o cálculo é feito em kgf/mm², mas foi transformado para MPa

<IPython.core.display.Latex object>

### Para chegar na potência, se faz necessário o cálculo das forças e dos momentos 

A força, na laminação a quente, é calculada pela área de contato e pela pressão específica $K_w$.

$K_w$ depende de dois novos fatores, a velocidade de deformação $\dot{\phi}$ e o coeficiente de plasticidade $\eta$:

In [17]:
%%render
#long

eta = 0.01*(14-0.01*Temp_processo)*s #por questões de consistência dimensional, trata-se $\eta$ como $s^{-1}$

omega_angular = V_linear/Raio

<IPython.core.display.Latex object>

In [18]:
%%render
#short

phi_dot = (2*V_linear*(Delta_h/Raio)**0.5)/(h_i+h_f)

<IPython.core.display.Latex object>

Finalmente, $K_w$:

In [19]:
%%render
#long

K_w = (1+(1.6*mu*l_d-1.2*Delta_h)/(h_i+h_f))*(sigma_e/(9.81*MPa)+eta*phi_dot)*(9.81*MPa) #como o cálculo é empírico e envolve kgf/mm², alguma complicação no dimensional acontece

<IPython.core.display.Latex object>

Perto do fim, as Forças:

In [20]:
%%render
#short

F = K_w*A_c

<IPython.core.display.Latex object>

### E, para finalizar, os Momentos e a Potência:

In [23]:
%%render
#long

M = F * l_d

eta_mecanica = 0.85

P = ((M * omega_angular)/eta_mecanica)*(1/(0.735*1000*W)) #CV

<IPython.core.display.Latex object>